In [1]:
%matplotlib widget
import netCDF4
import pyhdf.SD
from osgeo import gdal
import numpy
import geopandas
import pyproj
import rasterio
import rasterio.features
import shapely
import matplotlib

In [2]:
tile_name = 'data/MOD09GA.A2021309.h08v05.061.2021311034604.hdf'
group = 'MODIS_Grid_500m_2D'
ds = 'sur_refl_b01_1'

# User

In [3]:
left = -11119505.1966670006513596
right= -10007554.6769999992102385
bottom = 3335851.5589999998919666
top = 4447802.0786669999361038

# rasterio

In [4]:
ds_trunk = 'HDF4_EOS:EOS_GRID:{file}:{group}:{ds}'
ds_string = ds_trunk.format(file=tile_name, group=group, ds=ds)
data = rasterio.open(ds_string, 'r') 

In [5]:
data.crs
data.res

(463.31271652791725, 463.3127165279167)

In [6]:
left = data.bounds.left
right = data.bounds.right
xres = data.width

bottom = data.bounds.bottom
top = data.bounds.top
yres = data.height

# pyhdf + GDAL

In [7]:
tile_sd = pyhdf.SD.SD(tile_name)
tile_gdal = gdal.Open(ds_string, gdal.GA_ReadOnly)
metadata = tile_gdal.GetMetadata()

In [8]:
trans = tile_gdal.GetGeoTransform()
tile_gdal.GetProjection()
tile_gdal.RasterXSize
tile_gdal.ReadAsArray()

array([[  -8, -100,  -26, ..., 3429, 3536, 3466],
       [ -15,  -15,  -16, ..., 3429, 3629, 3631],
       [ -13,  -16,  -16, ..., 3593, 3593, 3536],
       ...,
       [1058, 1160, 1156, ...,  765,  958,  958],
       [1150, 1160, 1156, ...,  878,  878,  782],
       [1082, 1082, 1146, ...,  878,  782, 1029]], dtype=int16)

In [9]:
height = trans[5]
width = trans[1]

left = trans[0]
right = left + tile_gdal.RasterXSize*width
top = trans[3]
bottom = top + tile_gdal.RasterYSize*height

# Centers

In [11]:
xs = numpy.tile(numpy.arange(left, right, width), (2400, 1)).T
ys = numpy.tile(numpy.arange(top, bottom, height), (2400, 1))

In [24]:
type(centers)

list

In [19]:
#xs.flatten()
centers = geopandas.points_from_xy(xs.flatten(), ys.flatten())


In [16]:
cells = []
centers = []

for x in xs.flatten():
    for y in ys.flatten():
        center = shapely.geometry.polygon.Point(x+width/2, y+height/2)
        centers.append(center)
        #cell = shapely.geometry.polygon.Polygon([(x, y), (x+width, y),(x+width, y+height),(x, y+height),(x, y)])
        #cells.append(cell)

KeyboardInterrupt: 

In [ ]:
ix = numpy.numpy.tile(numpy.arange(0, 2400), (2400,1)).T
iy = numpy.numpy.tile(numpy.arange(0, 2400), (2400,1))

In [ ]:
x

In [ ]:
df = geopandas.GeoDataFrame({'centers': centers, 
                             'cells': cells,
                             'x': xs.flatten(),
                             'y': ys.flatten(),
                             'ix': ix.flatten(),
                             'iy': iy.flatten()})

In [ ]:
df

In [ ]:
wkt = '''PROJCS["example_name",
            GEOGCS["GCS_Undefined",
                DATUM["Undefined",SPHEROID["User_Defined_Spheroid",637007.181,0.0]],        
                UNIT["Degree",0.0174532925199433]],
            PROJECTION["Sinusoidal"],            
            UNIT["Meter",1.0]]'''
crs = pyproj.crs.CRS(wkt)
points = geopandas.points_from_xy(xs.flatten()[0:5000], ys.flatten()[0:5000])
gdf = geopandas.GeoDataFrame(geometry=points, crs=crs)
gdf.to_file('pts.gpkg')

# Make Grid

In [ ]:
lat = ys *  90 / 10000000 
lon = xs / numpy.cos(numpy.radians(lat)) *  90 / 10000000

#lat = ys / 10000000 / 0.0174532925199433 * numpy.pi/2
#lon = xs / numpy.cos(numpy.radians(lat)) / 10000000 / 0.0174532925199433  * numpy.pi/2


In [ ]:
wkt = '''GEOGCS["GCS_Undefined",
            DATUM["Undefined",SPHEROID["User_Defined_Spheroid",6371007.181,0.0]],        
            UNIT["Degree",0.0174532925199433]],'''
crs = pyproj.crs.CRS(wkt)
points = geopandas.points_from_xy(lon.flatten()[0:5000], lat.flatten()[0:5000])
#gdf = geopandas.GeoDataFrame(geometry=points, crs='EPSG:4326')
gdf = geopandas.GeoDataFrame(geometry=points, crs=crs)
gdf.to_file('pts.gpkg')
gdf

# MOD09GA

In [ ]:
sur_refl_b01_1 = mod09ga.select('sur_refl_b01_1').get()
sur_refl_b02_1 = mod09ga.select('sur_refl_b02_1').get()
sur_refl_b03_1 = mod09ga.select('sur_refl_b03_1').get()
sur_refl_b04_1 = mod09ga.select('sur_refl_b04_1').get()
sur_refl_b05_1 = mod09ga.select('sur_refl_b05_1').get()
sur_refl_b06_1 = mod09ga.select('sur_refl_b06_1').get()
sur_refl_b07_1 = mod09ga.select('sur_refl_b07_1').get()

# MOD09

In [ ]:
dataset = gdal.Open(filename, gdal.GA_ReadOnly)
